In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import sys
import time
import threading
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser


from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-02-17 13:45:34,428 - Authenticating...
2023-02-17 13:45:35,077 - Successfully authenticated


In [5]:
super_clarion(clarion)
runner = Runner(clarion)

In [6]:
### check metrics
pd.concat(clarion.metrics()).to_clipboard()

### Metrics

In [7]:
pv = clarion.metric('pv', 'PresentValue', 'Any')
local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any')
strike = clarion.metric('strike', 'Strike', 'Any')
notional = clarion.metric('notional', 'Notional', 'Any')
expiry = clarion.metric('expiry', 'ExpiryDate', 'Any')
maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'})
start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'})
fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates')
delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'})
theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'})
price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'})
price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'})
price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'})
price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'})
spot = clarion.metric('spot', 'Spot', 'Rates')
vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'})
atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'})
atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'})
strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'})
strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'})
carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'})

metrics = [
    pv,
    local_pv,
    strike,
    notional,
    expiry,
    maturity,
    start,
    fwd_rate,
    spot,
    delta,
    theta,
    price_cents,
    price_bps,
    price_pct,
    price_unit,
    # vega,
    # atm_vol_day,
    atm_vol_ann,
    # strike_vol_day,
    strike_vol_ann,
    # carry_day
]

### Class

In [38]:
sep = '__'


class Pricer:
    
    def get_data(self, clarion_trades, targets, start_date=None):
        start_date = start_date or today() + BDay() - BDay()
        all_trades = []
        i = 0
        for key, val in clarion_trades.items():
            if isinstance(val, dict):
                for key2, trade in val.items():
                    trade.id = f"{key}{sep}{key2}"
                    all_trades.append(trade)
                    i += 1
            elif isinstance(val, list):
                for trade in val:
                    trade.id = f"{key}{sep}{i}"
                    all_trades.append(trade)
                    i += 1
        self.trades = all_trades
        if today() + BDay() - BDay() != today():
            start_date = start_date or today() + BDay() - BDay()
            df = runner.run(metrics, all_trades, start_date=start_date)   
        else:
            df = runner.run(metrics, all_trades)
        
        # df = df.set_index('id').loc[[t.id  for t in all trades]]
        res = pd.concat([pd.DataFrame(df['id'].str.split(sep).to_list(), columns=['trade', 'id']), runner.df.drop('id', axis=1)], axis=1)
        col_order = [*res.columns[~res.columns.isin([metric.label for metric in metrics])], *[metric.label for metric in metrics]]
        res['notional'] = res['notional'].astype(float)
        res = res.loc[:, col_order]
        res = res.set_index('trade').loc[[*trades], :].reset_index().replace('', np.nan).infer_objects()
        self.data = res
        
        ress = []
        for index, group in res.groupby('trade', sort=False):
            # display(group)
            
            target_mult = targets.get(index, {}).get('mult', {})
            totals = pd.Series(
                {
                    'trade': index,
                    'id': 'total',
                    'price_bps': (group['price_bps'] * group['notional']).sum() / group['notional'].abs().iloc[0],
                    'price_unit': (group['price_unit'] * group['notional']).sum() / group['notional'].abs().iloc[0],
                    'size': ' x '.join([millify(x) for x in group['notional']]),
                    'pv': group['pv'].sum(),
                    'local_pv': group['local_pv'].sum(),
                    'delta': group['delta'].sum(),
                    'fwd_rate': (group['fwd_rate'] * np.sign(group['notional'])).sum() if not target_mult.get("fwd_rate") else (group['fwd_rate'] * np.sign(group['delta']) * np.asarray(target_mult.get("fwd_rate"))).sum(),
                }
            ).to_frame().T
            r = pd.concat([totals, group])[['size', *col_order]]
            ress.append(r)
        ress = pd.concat(ress)
        
        self.res = ress.infer_objects().reset_index(drop=True).drop('period', axis=1)
        total = ress.query('id=="total"').dropna(axis=1).drop("id", axis=1).set_index('trade')
        # ress._total = total
        
        targets_ = pd.DataFrame(targets).T
        if 'mult' in targets_:
            targets_ = targets_.drop('mult', axis=1)
        exprs = {}
        for col in total:
            exprs[col] = bh.bhTsCalcExpression(
                resultnames=range(len(targets_)),  # if omitted it doesn't work with multiple expressions at once, 
                expressions=[*targets_.index], 
                varnamesorhndl=[*total.index], 
                varvalues=total[col]
            )
        exprs = pd.DataFrame(exprs, index = targets_.index)
        
        
        watch = pd.concat([exprs, targets_], axis=1) # total.merge(exprs, right_index=True, left_index=True, how='right')
        # display(exprs)
        # display(total)
        # display(watch)
        
        watch = watch.assign(breach=watch.apply(lambda row: eval(f"{row[row['var']]} {row['op']} {row['target']}") if not pd.isnull(row['target']) else False, axis=1))
        self.watch = watch.infer_objects()

In [9]:
def highlight_cond(df, props, col):
    d = pd.DataFrame(None, df.index, df.columns)
    d.loc[df[col]] = props
    return d 

### Trades

In [48]:
trades = dict(
    eur_bear_flat_9m1y_9m2y = [
        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 3M',
            expiry='9m',
            tenor='1y',
            strike='3.65',
            swaption_type='Pay',
            notional=2e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),
        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike='3.4',
            swaption_type='Pay',
            notional=-1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),

        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike='2.8',
            swaption_type='Rec',
            notional=1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),

        clarion.instruments.RatesSwaption(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            expiry='9m',
            tenor='2y',
            strike='2.0',
            swaption_type='Rec',
            notional=-1e9,
            settle_type='Cash Price',
            collateral_currency='USD',
            pay_settle='Spot',
            #id='eur_9m_bear_flat'
        ),
    ],
    eur3m_bull_steep_15m1y_15m1y1y = [
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 3M',
                expiry='2024-05-03',
                tenor='1y',
                strike='2.425',
                swaption_type='Rec',
                notional=1e9,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 3M',
                expiry='2024-05-03',
                tenor='1yx1y',
                strike='2.0',
                swaption_type='Rec',
                notional=-1e9,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    eur6m_bull_steep_30m2y_30m7y = [
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 6M',
                expiry='30m',
                tenor='2y',
                strike='1.8',
                swaption_type='Rec',
                notional=400e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='EUR',
                instrument_name='EURIBOR A 6M',
                expiry='30m',
                tenor='7y',
                strike='1.8',
                swaption_type='Rec',
                notional=-120e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    sofr_bull_steep_2y7y_2y30y = [
            clarion.instruments.RatesSwaption(
                currency='USD',
                instrument_name='SOFR A',
                expiry='2y',
                tenor='7y',
                strike='2.3',
                swaption_type='Rec',
                notional=160e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='USD',
                instrument_name='SOFR A',
                expiry='2y',
                tenor='30y',
                strike='2.4',
                swaption_type='Rec',
                notional=-36e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    sonia_rec_fly_18m1y = [
            clarion.instruments.RatesSwaption(
                currency='GBP',
                instrument_name='SONIA A',
                expiry='18m',
                tenor='1y',
                strike='2.5',
                swaption_type='Rec',
                notional=800e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='GBP',
                instrument_name='SONIA A',
                expiry='18m',
                tenor='1y',
                strike='1.5',
                swaption_type='Rec',
                notional=-1600e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='GBP',
                instrument_name='SONIA A',
                expiry='18m',
                tenor='1y',
                strike='1.0',
                swaption_type='Rec',
                notional=800e6,
                settle_type='Cash Price',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    eur6m_fly_2y2y_4y2y_6y2y = [
        clarion.instruments.RatesSwap(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            notional=256e6,
            strike='a',
            start_date='2y',
            tenor='2y',
            swap_type='Rec',
            clearing_house='LCH',
        ),
        clarion.instruments.RatesSwap(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
             notional=547e6,
            strike='a',
            start_date='4y',
            swap_type='Pay',
            tenor='2y',
            clearing_house='LCH',
        ),
        clarion.instruments.RatesSwap(
            currency='EUR',
            instrument_name='EURIBOR A 6M',
            notional=288e6,
            strike='a',
            start_date='6y',
            tenor='2y',
            swap_type='Rec',
            clearing_house='LCH',
        )
    ],
    jpy_payer_fly_1y10y = [
            clarion.instruments.RatesSwaption(
                currency='JPY',
                instrument_name='TONAR A',
                expiry='1y',
                tenor='10y',
                strike='1.25',
                swaption_type='Pay',
                notional=10e9,
                settle_type='Physical Cleared JSCC',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='JPY',
                instrument_name='TONAR A',
                expiry='1y',
                tenor='10y',
                strike='2.0',
                swaption_type='Pay',
                notional=-15e9,
                settle_type='Physical Cleared JSCC',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
            clarion.instruments.RatesSwaption(
                currency='JPY',
                instrument_name='TONAR A',
                expiry='1y',
                tenor='10y',
                strike='3.0',
                swaption_type='Pay',
                notional=5e9,
                settle_type='Physical Cleared JSCC',
                collateral_currency='USD',
                pay_settle='Spot',
                #id='eur_9m_bear_flat'
            ),
    ],
    spx_nov23_put_spread = [
        clarion.instruments.EquityCashOption(
            instrument_name='SPX',
            contract_type='US',
            expiry='2023-11-17',
            strike=3000,
            option_type='Put',
            exercise_type='European',
            quantity=600

        ),
        clarion.instruments.EquityCashOption(
            instrument_name='SPX',
            contract_type='US',
            expiry='2023-11-17',
            strike=2900,
            option_type='Put',
            exercise_type='European',
            quantity=-600

        )
    ]
)

### Targets

In [49]:
targets = {
    'eur_bear_flat_9m1y_9m2y': {
        "var": "local_pv",
        "op":  "<=", 
        "target": 600e3, 
        "notes": "we BUY this structure in 500m on the first leg for 175k or better"
    },
    'eur3m_bull_steep_15m1y_15m1y1y': {
        "var": "local_pv", 
        "op":  "<=",
        "target": -180e3,
        "notes": "we SELL 1y1y and receive 180k"
    },
    'eur6m_bull_steep_30m2y_30m7y': {
        "var": "local_pv", 
        "op":  "<=",
        "target": 0,
        "notes": "we SELL 7y pay and pay less than 50k in 400x120"
    },
    'sofr_bull_steep_2y7y_2y30y': {
        "var": "local_pv", 
        "op":  "<=",
        "target": 680e3,
        "notes": "we SELL 30y pay 680k"
    },
    'sonia_rec_fly_18m1y': {
        "var": "price_bps", 
        "op":  "<=",
        "target": 8,
        "notes": "we SELL 1.5 strike (middle) for 8.5 or better"
    },
    'spx_nov23_put_spread': {
        "var": "price_unit", 
        "op":  "<=",
        "target": 5.0,
        "notes": ""
    },
    # 'sofr_bull_steep_2y7y_2y30y - sofr_bull_steep_2y7y_2y30y': {
    #     # "var": "local_pv", 
    #     # "op":  ">=",
    #     # "target": 2e6,
    #     # "notes": "we sell 30y pay -640k"
    # },
    'eur6m_fly_2y2y_4y2y_6y2y': {
        "var": "fwd_rate", 
        "op":  "<=",
        "target": -0.215,
        "notes": "we PAY 19.375 or better",
        'mult': {"fwd_rate": (1, 2, 1)}
    },
    
    # 'jpy_payer_fly_1y10y': {
    #     "var": "price_bps", 
    #     "op":  "<=",
    #     "target": 6.5,
    #     "notes": "we PAY 7.3 or better",
    # },
    'jpy_payer_fly_1y10y': {
        "var": "pv", 
        "op":  "<=",
        "target": 475e3,
        "notes": "we PAY 7.3 or better",
    },
}
targets_ = pd.DataFrame(targets).T

2023-02-17 16:30:27,677 - Requesting data
2023-02-17 16:30:39,351 - Request complete


In [39]:
def fn(pricer, out):
    last_update = time.ctime()
    last_email_sent = time.time()
    while True:
        if event.is_set():
            break
        try:
            p.get_data(trades, targets)
            last_update = time.ctime()
            s2 = p.watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None).format(na_rep='').format(
                partial(millify, decimal=3, pct=None, zero_rep='-'), 
                subset=p.watch.columns[p.watch.dtypes == 'float64']).set_caption('Trades and triggers')
            s1 = p.res.style.format(na_rep='').format(
                partial(millify, decimal=3, pct=None, zero_rep='-'), 
                subset=p.res.columns[p.res.dtypes == 'float64']
            )
            indices = p.res.groupby('trade', sort=False).apply(lambda df: df.index[-1])
            s1 = s1.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')
            clear_output()
            # out.clear_output(wait=True)
            out.outputs = ()
            out.append_display_data(last_update)
            out.append_display_data(s2)
            out.append_display_data(s1)
            # display(out)

            if ((time.time() - last_email_sent) / 3600) >= 1:
                outlook = win32.Dispatch('outlook.application')
                mail = outlook.CreateItem(0)
                mail.To = 'menashe.banit@brevanhoward.com'
                # mail.To = 'giovambattista.perciaccante@brevanhoward.com'
                mail.Subject = f'QUICK UPDATE: Trade Watch {pd.Timestamp.now(): %d%b%y %H:%M}'
                file_name = 'trades'
                html_body = ''
                for i, s in enumerate([s2, s1]):
                    file_name_ = f"{file_name}{i}.png"
                    dfi.export(s, file_name_)
                    image_file = open(file_name_, 'rb').read()
                    encoded_image = base64.b64encode(image_file).decode("utf-8")
                    html_body += f'{"<br>" * (5 if i > 0 else 1)}<img src="data:image/png;base64,{encoded_image}"/>'
                    mail.Attachments.Add(f"{str(Path().resolve() / file_name_)}")
                mail.HTMLBody = html_body
                # mail.HTMLBody = d[1].reset_index().to_csv(header=False, index=False)
                # To attach a file to the email (optional)
                # attachment  = "Path to the attachment"
                mail.Send()
                last_email_sent = time.time()
            # break
            time.sleep(30)
        except:
            clear_output()
            display(f"BROKEN: {last_update}")
            display(s2)
            display(s1)
            time.sleep(5)

In [40]:
p = Pricer()
out = w.Output()

In [41]:
thread = threading.Thread(target=fn, kwargs={'pricer': p, 'out': out})
event = threading.Event()

In [42]:
thread.start()

In [43]:
out

Output()

2023-02-17 14:03:30,555 - Requesting data


In [ ]:
event.set()

### Send EMAIL

In [15]:
p.get_data(trades, targets)

2023-02-17 13:46:01,912 - Requesting data
2023-02-17 13:46:14,520 - Request complete
2023-02-17 13:46:15,113 - Response Count Summary - DFRAME:21, IS_ERROR:67


In [45]:
s2 = p.watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None).format(na_rep='').format(
            partial(millify, decimal=3, pct=None, zero_rep='-'), 
            subset=p.watch.columns[p.watch.dtypes == 'float64']).set_caption('Trades and triggers')
s1 = p.res.style.format(na_rep='').format(
    partial(millify, decimal=3, pct=None, zero_rep='-'), 
    subset=p.res.columns[p.res.dtypes == 'float64']
)
indices = p.res.groupby('trade', sort=False).apply(lambda df: df.index[-1])
s1 = s1.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')

In [46]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'menashe.banit@brevanhoward.com'
# mail.To = 'giovambattista.perciaccante@brevanhoward.com'
mail.Subject = f'QUICK UPDATE: Trade Watch {pd.Timestamp.now(): %d%b%y %H:%M}'
file_name = 'trades'
html_body = ''
for i, s in enumerate([s2, s1]):
    file_name_ = f"{file_name}{i}.png"
    dfi.export(s, file_name_)
    image_file = open(file_name_, 'rb').read()
    encoded_image = base64.b64encode(image_file).decode("utf-8")
    html_body += f'{"<br>" * (5 if i > 0 else 1)}<img src="data:image/png;base64,{encoded_image}"/>'
    mail.Attachments.Add(f"{str(Path().resolve() / file_name_)}")
mail.HTMLBody = html_body
# mail.HTMLBody = d[1].reset_index().to_csv(header=False, index=False)
# To attach a file to the email (optional)
# attachment  = "Path to the attachment"
mail.Send()